In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [44]:
lines= pd.read_table('simplewordstranslate.txt', names=['eng', 'tel'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [45]:
lines.shape

(1822, 2)

In [46]:
lines.eng=lines.eng.str.lower()#apply(lambda x: x.lower())
lines.tel=lines.tel.str.lower()

In [47]:
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', str(x)))
lines.tel=lines.tel.apply(lambda x: re.sub("'", '', str(x)))

In [48]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.tel=lines.tel.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [49]:
lines.eng

0               అంచనా
1              అంటాడు
2              అంటారు
3                అంటూ
4                అంటే
5              అంటేనే
6               అండగా
7               అండ్‌
8               అంతకు
9               అంతగా
10            అంతర్గత
11         అంతర్గతంగా
12               అంతా
13               అంతే
14            అందరికీ
15              అందరూ
16              అందాక
17             అందాలు
18               అంది
19       అకస్మాత్తుగా
20              అక్కడ
21           అక్కడికీ
22             అక్కడే
23        అజాగ్రత్తలు
24                అటూ
25           అడపాదడపా
26            అడిగాడు
27            అడిగారు
28              అడిగి
29           అడిగింది
            ...      
1792      స్థాపించాడు
1793         స్థాయిని
1794         స్థితిలో
1795           స్నేహం
1796       స్నేహితులు
1797         స్పందనకు
1798    స్పర్శనిబట్టి
1799            స్పృహ
1800       స్ఫూర్తిని
1801         స్రావాలు
1802            స్వంత
1803      స్వభావానికి
1804            స్వయం
1805          స్వరంతో
1806      

In [50]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.tel = lines.tel.apply(lambda x: re.sub("[0123456789]", "", x))

In [51]:
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.tel=lines.tel.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.tel=lines.tel.apply(lambda x: re.sub(" +", " ", x))

lines.tel = lines.tel.apply(lambda x : 'S'+ x + 'E')

In [52]:
lines.sample(10)

,eng,tel
1518,వచ్చేవారు,Sవచ్చుE
931,ద్వారా,Sద్వారాE
367,ఒరేయ్,Sఒరేయ్E
487,కొడుకు,SకొడుకుE
1078,పనీ,SపనిE
840,తెచ్చుకుంటూ,Sతెచ్చుE
1578,విజేతలకే,SవిజేతE
588,చదువు,SచదువుE
997,నిర్ణయాన్నీ,Sనిర్ణయంE
523,క్షేమంగా,Sక్షేమంE


In [53]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng:
        if word not in all_eng_words:
            all_eng_words.add(word)
all_eng_words

{'ం',
 'అ',
 'ఆ',
 'ఇ',
 'ఈ',
 'ఉ',
 'ఊ',
 'ఎ',
 'ఏ',
 'ఐ',
 'ఒ',
 'ఓ',
 'ఔ',
 'క',
 'ఖ',
 'గ',
 'ఘ',
 'చ',
 'ఛ',
 'జ',
 'ఞ',
 'ట',
 'ఠ',
 'డ',
 'ణ',
 'త',
 'థ',
 'ద',
 'ధ',
 'న',
 'ప',
 'ఫ',
 'బ',
 'భ',
 'మ',
 'య',
 'ర',
 'ల',
 'ళ',
 'వ',
 'శ',
 'ష',
 'స',
 'హ',
 'ా',
 'ి',
 'ీ',
 'ు',
 'ూ',
 'ృ',
 'ె',
 'ే',
 'ై',
 'ొ',
 'ో',
 'ౌ',
 '్',
 'ౖ',
 '\u200c'}

In [54]:
all_Telugu_words=set()
for tel in lines.tel:
    for word in tel:
        if word not in all_Telugu_words:
            all_Telugu_words.add(word)
all_Telugu_words

{'E',
 'S',
 'ం',
 'అ',
 'ఆ',
 'ఇ',
 'ఈ',
 'ఉ',
 'ఊ',
 'ఎ',
 'ఏ',
 'ఐ',
 'ఒ',
 'ఓ',
 'ఔ',
 'క',
 'ఖ',
 'గ',
 'ఘ',
 'చ',
 'ఛ',
 'జ',
 'ఞ',
 'ట',
 'ఠ',
 'డ',
 'ణ',
 'త',
 'థ',
 'ద',
 'ధ',
 'న',
 'ప',
 'ఫ',
 'బ',
 'భ',
 'మ',
 'య',
 'ర',
 'ల',
 'ళ',
 'వ',
 'శ',
 'ష',
 'స',
 'హ',
 'ా',
 'ి',
 'ీ',
 'ు',
 'ూ',
 'ృ',
 'ె',
 'ే',
 'ై',
 'ొ',
 'ో',
 'ౌ',
 '్',
 'ౖ',
 '\u200c'}

In [55]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l))
max_length_src = np.max(lenght_list)
print(max_length_src)

18


In [56]:
lenght_list=[]
for l in lines.tel:
    lenght_list.append(len(l))
max_length_tar = np.max(lenght_list)

print(max_length_tar)

17


In [57]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_Telugu_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_Telugu_words)
print(num_encoder_tokens, num_decoder_tokens)
num_decoder_tokens += 1

59 61


In [58]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [59]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [60]:
lines = shuffle(lines)
print(lines.head(10))

                eng         tel
46           అత్యాశ    Sఅత్యాశE
1322        మనుషులం     SమనిషిE
1331   మలచుకోగలగాలి      SమలచుE
1788      స్కేల్‌తో   Sస్కేల్‌E
669   చెల్లెలికన్నా  Sచెల్లెలుE
169           ఆయుధం     SఆయుధంE
991         నిరంతరం  SనిరంతరముE
396           కలిసి     SకలువుE
1215        ప్రాంతం   Sప్రాంతంE
837         తెగింపు   SతెగించుE


In [61]:
X, y = lines.eng, lines.tel
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
print(X_train.shape, X_test.shape)

(1639,) (183,)


In [62]:
X_train.to_pickle('Weights_Tel/X_train.pkl')
X_test.to_pickle('Weights_Tel/X_test.pkl')

In [63]:
def generate_batch(X = X_train, y = y_train, batch_size = 10):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src+1),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar+1),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar+1, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text):
                    if t<len(target_text)-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [64]:
latent_dim = 50

In [65]:

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [66]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [67]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [68]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 10
epochs = 50

train_samples,val_samples

(1639, 183)

In [69]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = int(train_samples/batch_size),
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = int(val_samples/batch_size))


Epoch 1/50
163/163 [==============================] - 14s 84ms/step - loss: 3.1971 - acc: 0.1933 - val_loss: 2.9690 - val_acc: 0.2347
Epoch 2/50
163/163 [==============================] - 9s 58ms/step - loss: 2.8703 - acc: 0.2634 - val_loss: 2.7875 - val_acc: 0.2722
Epoch 3/50
163/163 [==============================] - 11s 65ms/step - loss: 2.6531 - acc: 0.3118 - val_loss: 2.5484 - val_acc: 0.3293
Epoch 4/50
163/163 [==============================] - 11s 65ms/step - loss: 2.4764 - acc: 0.3413 - val_loss: 2.4281 - val_acc: 0.3501
Epoch 5/50
163/163 [==============================] - 7s 46ms/step - loss: 2.3231 - acc: 0.3783 - val_loss: 2.2841 - val_acc: 0.3842
Epoch 6/50
163/163 [==============================] - 15s 92ms/step - loss: 2.1775 - acc: 0.4101 - val_loss: 2.1704 - val_acc: 0.4035
Epoch 7/50
163/163 [==============================] - 12s 73ms/step - loss: 2.0462 - acc: 0.4441 - val_loss: 2.0509 - val_acc: 0.4412
Epoch 8/50
163/163 [==============================] - 12s 73ms/s

In [71]:
model.save_weights('nmt_weights.h5')

In [72]:
model.load_weights('nmt_weights.h5')

In [73]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [74]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    print(target_token_index)
    target_seq[0, 0] = target_token_index['S']
   

    print(target_seq)
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'E'  or len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        #print(sampled_token_index)
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [77]:
train_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [78]:
count=0
while(k<182):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    inp=X_test[k:k+1].values[0]
    act=y_test[k:k+1].values[0][1:-1]
    pred=decoded_sentence[:-1]
    print('Input English sentence:', inp)
    print('Actual Tel Translation:',act )
    print('Predicted Tel Translation:', pred)
    
    if(act==pred):
        count+=1
count

{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41, 'వ': 42, 'శ': 43, 'ష': 44, 'స': 45, 'హ': 46, 'ా': 47, 'ి': 48, 'ీ': 49, 'ు': 50, 'ూ': 51, 'ృ': 52, 'ె': 53, 'ే': 54, 'ై': 55, 'ొ': 56, 'ో': 57, 'ౌ': 58, '్': 59, 'ౖ': 60, '\u200c': 61}
[[2.]]
Input English sentence: సంతోషం
Actual Tel Translation: సంతోషం
Predicted Tel Translation: సంతోషం
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40,

Input English sentence: ఏదేదో
Actual Tel Translation: ఏది
Predicted Tel Translation: ఏది
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41, 'వ': 42, 'శ': 43, 'ష': 44, 'స': 45, 'హ': 46, 'ా': 47, 'ి': 48, 'ీ': 49, 'ు': 50, 'ూ': 51, 'ృ': 52, 'ె': 53, 'ే': 54, 'ై': 55, 'ొ': 56, 'ో': 57, 'ౌ': 58, '్': 59, 'ౖ': 60, '\u200c': 61}
[[2.]]
Input English sentence: బస్సుకి
Actual Tel Translation: బస్సు
Predicted Tel Translation: బస్సుక
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 

Input English sentence: చాలాసార్లు
Actual Tel Translation: చాలా
Predicted Tel Translation: చాలాందికోవడా
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41, 'వ': 42, 'శ': 43, 'ష': 44, 'స': 45, 'హ': 46, 'ా': 47, 'ి': 48, 'ీ': 49, 'ు': 50, 'ూ': 51, 'ృ': 52, 'ె': 53, 'ే': 54, 'ై': 55, 'ొ': 56, 'ో': 57, 'ౌ': 58, '్': 59, 'ౖ': 60, '\u200c': 61}
[[2.]]
Input English sentence: చదువు
Actual Tel Translation: చదువు
Predicted Tel Translation: చదువు
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 2

{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41, 'వ': 42, 'శ': 43, 'ష': 44, 'స': 45, 'హ': 46, 'ా': 47, 'ి': 48, 'ీ': 49, 'ు': 50, 'ూ': 51, 'ృ': 52, 'ె': 53, 'ే': 54, 'ై': 55, 'ొ': 56, 'ో': 57, 'ౌ': 58, '్': 59, 'ౖ': 60, '\u200c': 61}
[[2.]]
Input English sentence: ఆలోచించకుండా
Actual Tel Translation: ఆలోచన
Predicted Tel Translation: ఆలోచన
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల':

{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41, 'వ': 42, 'శ': 43, 'ష': 44, 'స': 45, 'హ': 46, 'ా': 47, 'ి': 48, 'ీ': 49, 'ు': 50, 'ూ': 51, 'ృ': 52, 'ె': 53, 'ే': 54, 'ై': 55, 'ొ': 56, 'ో': 57, 'ౌ': 58, '్': 59, 'ౖ': 60, '\u200c': 61}
[[2.]]
Input English sentence: వచ్చాడు
Actual Tel Translation: వచ్చి
Predicted Tel Translation: వచ్చు
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 

...

Input English sentence: ఉపన్యాసాలు
Actual Tel Translation: ఉపన్యాసం
Predicted Tel Translation: ఉపాసూధయ
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41, 'వ': 42, 'శ': 43, 'ష': 44, 'స': 45, 'హ': 46, 'ా': 47, 'ి': 48, 'ీ': 49, 'ు': 50, 'ూ': 51, 'ృ': 52, 'ె': 53, 'ే': 54, 'ై': 55, 'ొ': 56, 'ో': 57, 'ౌ': 58, '్': 59, 'ౖ': 60, '\u200c': 61}
[[2.]]
Input English sentence: తండ్రిని
Actual Tel Translation: తండ్రి
Predicted Tel Translation: తండురు
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ

{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41, 'వ': 42, 'శ': 43, 'ష': 44, 'స': 45, 'హ': 46, 'ా': 47, 'ి': 48, 'ీ': 49, 'ు': 50, 'ూ': 51, 'ృ': 52, 'ె': 53, 'ే': 54, 'ై': 55, 'ొ': 56, 'ో': 57, 'ౌ': 58, '్': 59, 'ౖ': 60, '\u200c': 61}
[[2.]]
Input English sentence: బాధ
Actual Tel Translation: బాధ
Predicted Tel Translation: బాధ
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41,

Input English sentence: ఒకళ్ల
Actual Tel Translation: ఒక
Predicted Tel Translation: ఒక
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41, 'వ': 42, 'శ': 43, 'ష': 44, 'స': 45, 'హ': 46, 'ా': 47, 'ి': 48, 'ీ': 49, 'ు': 50, 'ూ': 51, 'ృ': 52, 'ె': 53, 'ే': 54, 'ై': 55, 'ొ': 56, 'ో': 57, 'ౌ': 58, '్': 59, 'ౖ': 60, '\u200c': 61}
[[2.]]
Input English sentence: అసలే
Actual Tel Translation: అసలు
Predicted Tel Translation: అసలు
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31,

Input English sentence: మరి
Actual Tel Translation: మరి
Predicted Tel Translation: మరి
{'E': 1, 'S': 2, 'ం': 3, 'అ': 4, 'ఆ': 5, 'ఇ': 6, 'ఈ': 7, 'ఉ': 8, 'ఊ': 9, 'ఎ': 10, 'ఏ': 11, 'ఐ': 12, 'ఒ': 13, 'ఓ': 14, 'ఔ': 15, 'క': 16, 'ఖ': 17, 'గ': 18, 'ఘ': 19, 'చ': 20, 'ఛ': 21, 'జ': 22, 'ఞ': 23, 'ట': 24, 'ఠ': 25, 'డ': 26, 'ణ': 27, 'త': 28, 'థ': 29, 'ద': 30, 'ధ': 31, 'న': 32, 'ప': 33, 'ఫ': 34, 'బ': 35, 'భ': 36, 'మ': 37, 'య': 38, 'ర': 39, 'ల': 40, 'ళ': 41, 'వ': 42, 'శ': 43, 'ష': 44, 'స': 45, 'హ': 46, 'ా': 47, 'ి': 48, 'ీ': 49, 'ు': 50, 'ూ': 51, 'ృ': 52, 'ె': 53, 'ే': 54, 'ై': 55, 'ొ': 56, 'ో': 57, 'ౌ': 58, '్': 59, 'ౖ': 60, '\u200c': 61}
[[2.]]
Input English sentence: చప్పట్లు
Actual Tel Translation: చప్పట్లు
Predicted Tel Translation: చప్పు


64